# SI 618 - Homework #2: More Data Manipulation

# Background
You're a Data Science Consultant for an eCommerce retail company, they've asked you to analyze their sales database. Unfortunately, they did nothing to prepare or clean their data, only exporting their 3 database tables as JSON files. It's up to you to clean their data, analyze it and answer questions to help drive business value!

### The below files have been provided via the URLs shown:
- invoices.json https://github.com/umsi-data-science/data/raw/main/invoices.json
- items.json https://github.com/umsi-data-science/data/raw/main/items.json
- purchases.json https://github.com/umsi-data-science/data/raw/main/purchases.json

### They provided this data dictionary:

**InvoiceNo:** Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction.  
**StockCode:** Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.  
**Description:** Product (item) name. Nominal.   
**Quantity:** The quantities of each product (item) per transaction. Numeric.  
**InvoiceDate:** Invoice Date and time. Numeric, the day and time when each transaction was generated.  
**UnitPrice:** Unit price. Numeric, Product price per unit in sterling.  
**CustomerID:** Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.  
**Country:** Country name. Nominal, the name of the country where each customer resides.  

### A few notes from the company:
* If the InvoiceNo starts with the letter 'c', it indicates a cancellation. When conducting this analysis we only want to analyze invoices that were shipped. (ie. not canceled)
* The datasets should be able to be merged, each row in the invoice table corresponds to multiple rows in the purchases table.
* To find out the description or unit cost of an item in the purchase table, the StockCode should be used to match up the product in the items table. 
* They mentioned that they've been having a difficult time lately joining the items and purchases table, maybe there's something wrong with the columns?

### The total score for this assignment will be 100 points, consisting of:
- 10 pt: Overall quality of spelling, grammar, punctuation, etc. of written sentences. ([Guide](https://drive.google.com/file/d/1L0P7xJwjUGBvyb49mL3dw1Bt7hzRTiTl/view ))
- 10 pt: Codes are written in [PEP 8](https://www.python.org/dev/peps/pep-0008/) style.
- 80 pt: Homework questions. 

Version 2024.01.23.2.CT

# Answer the questions below.
- write your Python code that can answer the following questions 
- explain **ALL** your answers in plain English. 
- you can use as many code and markdown cells as you need for each question (i.e. don't limit yourself to just one of each if you feel you need more).

In [2]:
MY_UNIQNAME = 'jonasxie'  # replace this with your uniqname

### <span style="color:magenta"> Q1. [5 points] Describe the dataset. </span>
1. Load the data. 
1. How many total invoices have been placed?
1. How many unique customers are there?
1. What is the total number of unique items?
1. Are there any columns with null values?
1. Thinking ahead, how do you think you would join the different tables? Please share 2-3 sentences about your approach.

In [3]:
import requests
import pandas as pd

In [5]:
# load the json data from the link
DATA_INVOICES = pd.read_json("https://github.com/umsi-data-science/data/raw/main/invoices.json")
DATA_ITEMS = pd.read_json("https://raw.githubusercontent.com/umsi-data-science/data/main/items.json")
DATA_PURCHASES = pd.read_json("https://raw.githubusercontent.com/umsi-data-science/data/main/purchases.json")

In [6]:
# Describe the datasets to have a basic sense of the datasets
DATA_INVOICES.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25943 entries, 0 to 25942
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    25943 non-null  object 
 1   InvoiceDate  25943 non-null  object 
 2   CustomerID   22221 non-null  float64
 3   Country      25943 non-null  object 
dtypes: float64(1), object(3)
memory usage: 810.8+ KB


In [12]:
DATA_ITEMS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070 entries, 0 to 4069
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StockCode    4070 non-null   object 
 1   Description  3894 non-null   object 
 2   UnitPrice    4070 non-null   float64
dtypes: float64(1), object(2)
memory usage: 95.5+ KB


In [13]:
DATA_PURCHASES.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   InvoiceNo    541909 non-null  object
 1   StockCodeSC  541909 non-null  object
 2   Quantity     541909 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


In [8]:
# Calculate the number of the invoices
num_invoices = DATA_INVOICES.shape[0]
num_invoices

25943

**Q1.2**: There are 25943 invoices in the dataset.

In [11]:
# Q1.3 Calculate the number of unique customers
num_unique_customers = DATA_INVOICES['CustomerID'].nunique()
num_unique_customers

4372

**Q1.3**: There are 4372 unique customers in the dataset.

In [15]:
# Q1.4 Calculate the number of unique items
num_unique_items = DATA_ITEMS['StockCode'].nunique()
num_unique_items

4070

**Q1.4**: There are 4070 unique items in the dataset.

In [21]:
# Q1.5 Check the null values in each dataset
print("Missing values in Invoice Dataset:")
print(DATA_INVOICES.isnull().sum())
print("\n")
print("Missing values in Items Dataset:")
print(DATA_ITEMS.isnull().sum())
print("\n")
print("Missing values in Purchases Dataset:")
print(DATA_PURCHASES.isnull().sum())

Missing values in Invoice Dataset:
InvoiceNo         0
InvoiceDate       0
CustomerID     3722
Country           0
dtype: int64


Missing values in Items Dataset:
StockCode        0
Description    176
UnitPrice        0
dtype: int64


Missing values in Purchases Dataset:
InvoiceNo      0
StockCodeSC    0
Quantity       0
dtype: int64


**Q1.5**: There are 3722 null values in the `CustomerID` column of the invoice dataset. There are 176 null values in the `Description` column of the items dataset.

**Q1.6**: To join the datasets, I will first join the `Items` table to the `Purchases` by using the `StockCode` column in the `items` dataset and `StockCodeSC` in `Purchases` to match up products, add their `UnitPrice` and calculate the total spending for each product in the new `Purchase_Items` dataset. Then, I will use the `InvoiceNo` column in the `invoices` dataset to merge to the `Purchase_Items` dataset.

### <span style="color:magenta"> Q2. [10 points] Invoice Analysis </span>
1. For each customer calculate how many total invoices they have placed. List the top 10 customers who have placed an invoice in descending order.
2. Perform a similar calculation but instead of the number of invoices, calculate the total quantity of items ordered for each customer. List the top 10 customers in descending order.
3. Compare the top 10 customers, does it appear that the more invoices a customer have, the greater the total quantity of items? Explain your reasoning.

_Hint: For 2.2, you may need to join two datasets together to answer the question._

In [22]:
# Q2.1 List the top ten customers have placed most invoices
DATA_INVOICES['CustomerID'].value_counts().head(10)

CustomerID
14911.0    249
12748.0    225
17841.0    169
14606.0    130
13089.0    118
15311.0    118
12971.0     89
14527.0     86
13408.0     81
14646.0     77
Name: count, dtype: int64

In [24]:
# Q2.2 List the top ten customers have purchased the most items
## left join purchase to the invoice dataset on the invoice number
data_invoices_purchase = pd.merge(DATA_INVOICES, DATA_PURCHASES, on='InvoiceNo', how='left')

## calculate the number of items purchased by each customer and list the top ten customers
data_invoices_purchase['CustomerID'].value_counts().head(10)

CustomerID
17841.0    7983
14911.0    5942
14096.0    5128
12748.0    4731
14606.0    2840
15311.0    2491
14646.0    2085
13089.0    1857
13263.0    1677
14298.0    1640
Name: count, dtype: int64

**Q2.3**: The top 10 customers who have placed the most invoices are not the same as the top 10 customers with most items. This could be due to the fact that some customers place more orders with less items for each order, while others place fewer but larger orders. It weakens the correlation between the quantities of invoices placed and the quantities of items ordered.

### <span style="color:magenta"> Q3. [10 points] Item Analysis </span>
1. What is the average item-unit price? 
1. What % of items are under $25?
1. Generate a histogram of the unit prices. Select reasonable min/max values for the x-axis. Why did you pick those values? What do you notice about the histogram? 

In [15]:
# put your code here

(Use this space to explain your answers)

### <span style="color:magenta"> Q4. [25 points] Order Trends </span>
1. What are the top 10 most ordered items? Describe them. Do you see any trends?  
1. What are the top 5 invoices that generated the most revenue? (Revenue is calculated by "marking up" the unit price by 25%.) 
1. Do the top 5 invoices contain any of the top 10 most ordered items?

_Hint: When calculating the revenue we suggest adding a new column on the dataframe._

In [20]:
# put your code here

(Use this space to explain your answers)

### <span style="color:magenta"> Q5. [30 points] Customer Analysis </span>
1. Classify customers into segments based on the total revenue they have generated for the company.
 * low value: less than \$1750  
 * medium value: between 1750 and 8250 dollars
 * high value: greater than \$8250
1. How many customers are in each segment? 
1. Using the pivot table function, create a table that displays the average order quantity of each stock code for a given segment.
1. Are the items with the highest average order quantity generally the same across segments? Explain your reasoning. 
1. Choose three items and discuss any trends/differences you notice across the three segments.

_Hint: When calculating the segment, we suggest constructing a new dataframe as an intermediary step with the columns: CustomerID, Revenue, Segment._

In [105]:
# put your code here

(Use this space to explain your answers)

# Please submit your completed notebook in .IPYNB and .HTML formats via Canvas